In [ ]:
import os

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

from PIL import Image

import pickle
import numpy as np

import sharp
import h1data
import bq

In [ ]:
# Make folders
if not os.path.exists("fig"):
    os.mkdir("fig")
if not os.path.exists("fig/powerpoint"):
    os.mkdir("fig/powerpoint")
if not os.path.exists("fig/results"):
    os.mkdir("fig/results")

In [ ]:
fname_dump = 'sharp_res.pkl'

if os.path.exists(fname_dump):
    with open(fname_dump, 'rb') as f:
        s_list = pickle.load(f)
else:
    s_list = []
    datapath = os.path.join(os.getcwd(),"data")
    for dataset in os.listdir(datapath):
        full_path = os.path.join(datapath,dataset)
        if not "CaptureDL" in full_path:
            continue
     
        hdata = h1data.h1data(full_path)
        sdata = sharp.sharp.fromh1data(hdata)
    
        title = str()
        title = title.join([full_path.split("/")[-1].split("_")[2], " ", full_path.split("/")[-1].split("_")[0]])
    
        sdata.set_name(title)
        sdata.sharpen()
        sdata.evaluate()
        
        s_list.append(sdata)
    
    # Open a file for writing
    with open(fname_dump, 'wb') as f:
        pickle.dump(s_list, f)
        

In [ ]:
def normalize(arr):
  arr = arr - np.min(arr)  
  return arr / np.max(arr)

def create_moving_picture(filename, cube, fps=2, colormode=False):
    import imageio
    
    frames = []
    if colormode:
        for i in range(cube.shape[-1] - 40):
            frame = np.array(cube[:,:,[i+40,i+20,i]], dtype=np.uint8)
            frames.append(frame)
    else:
        for i in range(cube.shape[-1]):
            frame = np.array(cube[:,:,i], dtype=np.uint8)
            frames.append(frame)
        
        
    imageio.mimsave(filename, frames, fps=fps)


In [ ]:
create_moving_picture('fig/powerpoint/video.mp4', s_list[7].initial_cube,fps=7)
create_moving_picture('fig/powerpoint/video_sharpend.mp4', s_list[7].sharpened_cube,fps=7)

In [ ]:
h = h1data.h1data("data/20221107_CaptureDL_sudan_tl_2022_11_04T08_31_09")
wl = h.spec_coefficients
R_ind = np.argmin(abs(wl-600))
G_ind = np.argmin(abs(wl-553))
B_ind = np.argmin(abs(wl-500))

# Example image
ex_cube =  s_list[3] # <- Sudan

# Coordinates for crop in Sudan Image
x_start = 280
y_start = 80
x_end = 400
y_end = 300

In [ ]:
titles = []
for ind, sobj in enumerate(s_list):    
    titles.append(sobj.name)
    fname = sobj.name.replace(" ", "_")

    A = sobj.initial_cube[:,:,[R_ind, G_ind, B_ind]]
    A = np.array(A,dtype=np.uint8)
    im = Image.fromarray(A)
    im.save(f"fig/scenes/{fname}.png")

    A = sobj.sharpened_cube[:,:,[R_ind, G_ind, B_ind]]
    A = np.array(normalize(A)*255,dtype=np.uint8)
    im = Image.fromarray(A)
    im.save(f"fig/scenes/{fname}_sharpend.png")
    

In [ ]:
title = f"Raw Cube @ {wl[h.center_wavelength]:.0f}nm"
f = px.imshow(h.raw_cube[:,:,h.center_wavelength],
              labels=dict(x=title, color="DC Intensity"))
f.update_xaxes(side="top")
f.update_layout(margin=dict(l=0, r=0, t=0, b=0))
f.write_image("fig/powerpoint/raw_intensity.png", width=360, height = 380, scale=5)
f.show()

In [ ]:
title = f"Calibrated Cube @ {wl[h.center_wavelength]:.0f}nm"
f = px.imshow(h.l1a_cube[:,:,h.center_wavelength],
              labels=dict(x=title, color="~Radiance"))
f.update_xaxes(side="top")
f.update_layout(margin=dict(l=0, r=0, t=0, b=0))
f.write_image("fig/powerpoint/rad_intensity.png", width=360, height = 380, scale=5)
f.show()

In [ ]:
def normalize(arr):
  arr = arr - np.min(arr)  
  return arr / np.max(arr)

inspected_ind = [
    np.argmin(np.abs(wl-450)),
    np.argmin(np.abs(wl-553)),
    np.argmin(np.abs(wl-650)),
    np.argmin(np.abs(wl-750)),
]

sub_titles = []
for i, elem in enumerate(inspected_ind):
    sub_titles.append(f"{wl[elem]:.0f}nm")

fig = make_subplots(rows=len(inspected_ind), cols=1,
                   subplot_titles = sub_titles)

for i, elem in enumerate(inspected_ind):
    figure = px.imshow(np.rot90(ex_cube.initial_cube[:,:,elem]))
    for trace in range(len(figure["data"])):
        fig.append_trace(figure["data"][trace], row=1+i, col=1)

fig.update_layout(
    autosize=False,
    minreducedheight=i*200,
    height=i*250,
)
    
fig.update_layout(title_text=f"Radiance intensity for {ex_cube.name}")
fig.update_layout(margin=dict(l=0, r=0, b=0))
fig.write_image("fig/powerpoint/varying_wl_rad_intensity.png", width=600, height = 900, scale=3)
fig.show()


In [ ]:
new_titles = []
for e in sub_titles:
    new_titles.append("Sharpend " + e)

expand_sub_titles = []
for i,e in enumerate(2*sub_titles):
    if ((i+1) % 2 == 0):
        expand_sub_titles.append(new_titles[i//2])
    else:
        expand_sub_titles.append(sub_titles[i//2])
        

fig = make_subplots(rows=len(inspected_ind), cols=2,
                   subplot_titles = expand_sub_titles)

for i, elem in enumerate(inspected_ind):
    f1 = px.imshow(np.rot90(normalize(ex_cube.initial_cube[x_start:x_end,y_start:y_end,elem])))
    for trace in range(len(f1["data"])):
        fig.append_trace(f1["data"][trace], row=1+i, col=1)

    f2 = px.imshow(np.rot90(normalize(ex_cube.sharpened_cube[x_start:x_end,y_start:y_end,elem])))
    for trace in range(len(f2["data"])):
        fig.append_trace(f2["data"][trace], row=1+i, col=2)

fig.update_layout(
    autosize=False,
    minreducedheight=i*200,
    height=i*250,
)
    
fig.update_layout(title_text=f"Normalized Side-by-side zoomed for {ex_cube.name}")
fig.update_layout(margin=dict(l=0, r=0, b=0))
fig.write_image("fig/powerpoint/varying_wl_zoomed_sharp_v_orig_normalized.png", width=600, height = 900, scale=3)
fig.show()


In [ ]:
fig = make_subplots(rows=len(inspected_ind), cols=2,
                   subplot_titles = expand_sub_titles)

for i, elem in enumerate(inspected_ind):
    f1 = px.imshow(np.rot90(ex_cube.initial_cube[x_start:x_end,y_start:y_end,elem]))
    for trace in range(len(f1["data"])):
        fig.append_trace(f1["data"][trace], row=1+i, col=1)

    f2 = px.imshow(np.rot90(ex_cube.sharpened_cube[x_start:x_end,y_start:y_end,elem]))
    for trace in range(len(f2["data"])):
        fig.append_trace(f2["data"][trace], row=1+i, col=2)

fig.update_layout(
    autosize=False,
    minreducedheight=i*200,
    height=i*250,
)
    
fig.update_layout(title_text=f"Radiance Side-by-side zoomed for {ex_cube.name}")
fig.update_layout(margin=dict(l=0, r=0, b=0))
fig.write_image("fig/powerpoint/varying_wl_zoomed_sharp_v_orig_radiance.png", width=600, height = 900, scale=3)
fig.show()


In [ ]:
fig = make_subplots(rows=len(inspected_ind), cols=1,
                   subplot_titles = sub_titles)

for i, elem in enumerate(inspected_ind):
    figure = px.imshow(np.rot90(ex_cube.initial_cube[:,:,elem] - ex_cube.sharpened_cube[:,:,elem]))
    for trace in range(len(figure["data"])):
        fig.append_trace(figure["data"][trace], row=1+i, col=1)

fig.update_layout(
    autosize=False,
    minreducedheight=i*200,
    height=i*250,
)
    
fig.update_layout(title_text=f"Original v. sharpend intensity for {ex_cube.name}")
fig.update_layout(margin=dict(l=0, r=0, b=0))
fig.write_image("fig/powerpoint/varying_wl_orig_v_sharp_diff_intensity.png", width=600, height = 900, scale=3)
fig.show()

In [ ]:
bq_scores_original = []
bq_scores_sharpend = []
titles = []
for ind, sobj in enumerate(s_list):    
    titles.append(sobj.name)
    
    bq_original = sobj.brisque["initial"]    
    bq_scores_original.append(bq_original)
    
    bq_sharpend = sobj.brisque["sharpend"]    
    bq_scores_sharpend.append(bq_sharpend)
        
bq_scores_original = np.array(bq_scores_original)
bq_scores_sharpend = np.array(bq_scores_sharpend)

In [ ]:
stat_orig_fig = bq.plotScoreStatistics(wl[10:], bq_scores_original[:,10:] ,0.01)
stat_orig_fig.update_layout(
    title="BRISQUE statistics for calibrated cubes",
    xaxis_title="Wavelength (nanometer)",
    yaxis_title="BRISQUE Value",
)

stat_orig_fig.show()
fig.update_layout(margin=dict(l=0, r=0, b=0))
stat_orig_fig.write_image("fig/results/stat_orig_fig.pdf")
stat_orig_fig.write_image("fig/powerpoint/stat_orig_fig.png", width = 900, height = 500, scale = 2)

In [ ]:
stat_sharp_fig = bq.plotScoreStatistics(wl[10:], bq_scores_sharpend[:,10:] ,0.01)
stat_sharp_fig.update_layout(
    title="BRISQUE statistics for sharpend cubes",
    xaxis_title="Wavelength (nanometer)",
    yaxis_title="BRISQUE Value",
)

stat_sharp_fig.show()
fig.update_layout(margin=dict(l=0, r=0, b=0))
stat_sharp_fig.write_image("fig/results/stat_sharp_fig.pdf")
stat_sharp_fig.write_image("fig/powerpoint/stat_sharp_fig.png", width = 900, height = 500, scale = 2)

In [ ]:
all_orig_fig = bq.plotAllScores(wl[10:], bq_scores_original[:,10:] , titles)
all_orig_fig.update_layout(
    title="All BRISQUE scores for calibrated cubes",
    xaxis_title="Wavelength (nanometer)",
    yaxis_title="BRISQUE Value",
    legend_title="Scene Name"
)
all_orig_fig.show()
fig.update_layout(margin=dict(l=0, r=0, b=0))
all_orig_fig.write_image("fig/results/all_orig_fig.pdf")
all_orig_fig.write_image("fig/powerpoint/all_orig_fig.png", width = 900, height = 500, scale = 2)

In [ ]:
all_sharp_fig = bq.plotAllScores(wl[10:], bq_scores_sharpend[:,10:] , titles)
all_sharp_fig.update_layout(
    title="All BRISQUE scores for sharpend cubes",
    xaxis_title="Wavelength (nanometer)",
    yaxis_title="BRISQUE Value",
    legend_title="Scene Name"
)
all_sharp_fig.show()
fig.update_layout(margin=dict(l=0, r=0, b=0))
all_sharp_fig.write_image("fig/results/all_sharp_fig.pdf")
all_sharp_fig.write_image("fig/powerpoint/all_sharp_fig.png", width = 900, height = 500, scale = 2)

In [ ]:
diff = bq_scores_original - bq_scores_sharpend
all_diff_fig = bq.plotAllScores(wl[10:], diff[:,10:] , titles)
all_diff_fig.update_layout(
    title="Difference in BRISQUE score",
    xaxis_title="Wavelength (nanometer)",
    yaxis_title="BRISQUE Value Diffrence",
    legend_title="Scene Name"
)
all_diff_fig.show()
fig.update_layout(margin=dict(l=0, r=0, b=0))
all_diff_fig.write_image("fig/results/all_diff_fig.pdf")
all_diff_fig.write_image("fig/powerpoint/all_diff_fig.png", width = 900, height = 500, scale = 2)

## Intermidiate Steps
The plots below are intended to showcase intermidiate steps of the sharpening

In [ ]:
from numpy.linalg import svd
from skimage.exposure import match_histograms

image = ex_cube.initial_cube
sh_ind = ex_cube.sharpest_band_index

img_variable_form = np.reshape(image, (image.shape[0] * image.shape[1], image.shape[2]))
U, S, Vh = svd(img_variable_form, full_matrices=False)
principal_components = np.dot(U, np.diag(S))

component_cube = np.reshape(principal_components, (image.shape[0], image.shape[1], image.shape[2]))

matched_sharpest_band = match_histograms(image[:,:,47], component_cube[:,:,0])

In [ ]:

pcs_sub_titles = ["1st Principal Component", "2nd Principal Component", "3rd Principal Component"]
fig = make_subplots(rows=3, cols=1,
                   subplot_titles = pcs_sub_titles)

for i, elem in enumerate(pcs_sub_titles):
    figure = px.imshow(np.rot90(component_cube[:,:,i]))
    for trace in range(len(figure["data"])):
        fig.append_trace(figure["data"][trace], row=1+i, col=1)

fig.update_layout(
    autosize=False,
    minreducedheight=i*200,
    height=i*250,
)
    
fig.update_layout(title_text=f"Most significant principal components for {ex_cube.name}")
fig.update_layout(margin=dict(l=0, r=0, b=0))
fig.write_image("fig/powerpoint/most_significant_pcs.png", width=600, height = 800, scale=3)
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=3,
                   subplot_titles = [
                       "1st Principal Component",
                       "Matched Center Band",
                       "Difference Plot"
                   ])
plot_li = [
    component_cube[:,:,0],
    matched_sharpest_band,
    matched_sharpest_band - component_cube[:,:,0]
]
for i, elem in enumerate(plot_li):
    figure = px.imshow(elem)
    for trace in range(len(figure["data"])):
        fig.append_trace(figure["data"][trace], row=1, col=1+i)


fig.update_layout(title_text=f"Component substituion for {ex_cube.name}")
fig.update_layout(margin=dict(l=0, r=0, b=0))
fig.write_image("fig/powerpoint/pc_v_matched.png", width=900, height = 600, scale=3)
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=3,
                   subplot_titles = [
                       "1st Principal Component",
                       "Matched Center Band",
                       "Difference Plot"
                   ])

plot_li = [
    component_cube[x_start:x_end,y_start:y_end,0],
    matched_sharpest_band[x_start:x_end,y_start:y_end],
    matched_sharpest_band[x_start:x_end,y_start:y_end] - component_cube[x_start:x_end,y_start:y_end,0]
]
for i, elem in enumerate(plot_li):
    figure = px.imshow(elem)
    for trace in range(len(figure["data"])):
        fig.append_trace(figure["data"][trace], row=1, col=1+i)


fig.update_layout(title_text=f"Component substituion for zoomed in {ex_cube.name}")
fig.update_layout(margin=dict(l=0, r=0, b=0))
fig.write_image("fig/powerpoint/zoomed_pc_v_matched.png", width=900, height = 450, scale=3)
fig.show()